In [245]:
#load data
import pandas as pd
def load_data(csv_file, sample_fraction=1.0):
    """Loads Question Pairs from a CSV file

    Args:
        csv_file (str): Path to csv_file
        sample_fraction (float): Fraction of data to sample, default is 1.0
    Returns:
        tuple: A tuple containing supervised data pairs
        returns [],[] on error
    """
    sentences1 = []
    sentences2 = []
    is_duplicate = []
    
    try:
        df = pd.read_csv(csv_file, encoding='utf-8')
        #print("Column names:", df.columns)
        
        if sample_fraction < 1.0:
            df = df.sample(frac=sample_fraction, random_state=42).reset_index(drop=True)
        
        sentence1_series = df['question1']
        sentence2_series = df['question2']
        is_duplicate_series = df['is_duplicate']
        
        sentences1 = sentence1_series.tolist()
        sentences2 = sentence2_series.tolist()
        is_duplicate = is_duplicate_series.tolist()
        
        if len(sentences1) != len(sentences2):
            raise ValueError("The number of sentences in question1 and question2 do not match.")
        else:
            print(f"Loaded {len(sentences1)} sentences.")
        return sentences1, sentences2, is_duplicate
        
    except FileNotFoundError:
        print("Wrong Path")
        return [],[],[]
    
    except Exception as e:
        print(f"An {e} Error Occurred")
        return [],[],[]

DATA_PATH = r'C:/Users/Jash\Documents/Research\Semantic Equivilance\SemanticEquivilance/question_pairs/questions.csv'
sentences1, sentences2, value = load_data(DATA_PATH, sample_fraction=0.001)


Loaded 404 sentences.


In [338]:
import pennylane as qml
import numpy as np
from lambeq import AtomicType, BobcatParser, IQPAnsatz, Rewriter
from lambeq.backend.pennylane import to_pennylane as to_qml
import random

def get_circuit_state_vector(qml_qnode_func, num_wires):
    """
    Executes a PennyLane QNode function and returns the state vector.
    The qml_qnode_func should return a qml.QNode object.
    """
    dev = qml.device("default.qubit", wires=num_wires)

    @qml.qnode(dev)
    def state_vector_wrapper():
        qml_qnode_func() # Execute the actual circuit operations
        return qml.state()

    return state_vector_wrapper()

def swap_test(state1_vec, state2_vec, num_qubits):
    """
    Performs a Quantum Swap Test between two quantum state vectors.

    Args:
        state1_vec (np.ndarray): The first state vector.
        state2_vec (np.ndarray): The second state vector.

    Returns:
        float: The estimated squared overlap (fidelity) between the two states.
    """
    if 2**num_qubits != len(state1_vec):
        raise ValueError("State vectors must have a length that is a power of 2.")

    total_qubits = 1 + 2 * num_qubits #1 Ancilla qubit + 2 state qubits

    dev = qml.device("default.qubit", wires=total_qubits, shots=1000) # N = 1000 runs of the circuit for statistical significance

    @qml.qnode(dev)
    def circuit(): #|0 , psi, phi>
        # Step 1: Prepare the ancilla qubit in a superposition
        qml.Hadamard(wires=0)
        print(f"Comparing: {state1_vec} and {state2_vec}")
        # Step 2: Prepare the two input states
        qml.StatePrep(state1_vec, wires=range(1, 1 + num_qubits), normalize=True) #take another look at initial state prep

        qml.StatePrep(state2_vec, wires=range(1 + num_qubits, 1 + 2 * num_qubits))

        # Step 3: Apply controlled-SWAP gates
        for i in range(num_qubits):
            qml.CSWAP(wires=[0, 1 + i, 1 + num_qubits + i]) #selects every register of phi and psi for swap

        # Step 4: Apply Hadamard to ancilla
        qml.Hadamard(wires=0)
        # Step 5: Measure the ancilla qubit
        return qml.sample(wires=0)

    measurement_results = circuit()
    prob_0 = np.sum(measurement_results == 0) / len(measurement_results)
    squared_overlap = 2 * prob_0 - 1

    return squared_overlap

def lambeq_sentence_to_state_vector(sentence, ansatz, parser, rewriter, include_debug_prints=False):
    """
    Converts a natural language sentence into a quantum state vector
    using Lambeq's BobcatParser and IQPAnsatz, handling parameterization
    via PennyLaneModel.

    Args:
        sentence (str): The input sentence.
        ansatz (lambeq.ansatz.Ansatz): The quantum ansatz to apply.
        parser (lambeq.parser.Parser): The parser to convert sentence to diagram.

    Returns:
        tuple: A tuple containing the state vector (np.ndarray) and
               the number of qubits (int).
    """
    try:
        if include_debug_prints:
            print(f"\n--- Debugging: Sentence '{sentence}' ---")

        # Step 1: Convert sentence to a DisCoPy diagram
        diagram = parser.sentence2diagram(sentence)
        if include_debug_prints:
            print("Step 1: Sentence parsed to diagram.")

        # Step 2: Rewrite the diagram
        rewritten_diagram = rewriter(diagram)
        if include_debug_prints:
            print("Step 2: Diagram rewritten.")

        # Step 3: Normalize the diagram
        normalized_diagram = rewritten_diagram.normal_form()
        if include_debug_prints:
            print("Step 3: Diagram normalized.")
            
        # Step 4: Apply the ansatz to the normalized diagram to get a DisCoPy circuit
        circuit = ansatz(normalized_diagram)
        if include_debug_prints:
            print("Step 4: Ansatz applied to create DisCoPy circuit.")

        # Step 5: Convert the DisCoPy circuit to a PennyLane circuit object
        temp_qml_circuit = to_qml(circuit)
        if include_debug_prints:
            print("Step 5: DisCoPy circuit converted to PennyLane object.")

        num_qubits = temp_qml_circuit._n_qubits
        param_structure = temp_qml_circuit._params
        
        if include_debug_prints:
            print(f"Parameter structure: {param_structure}")
            print(f"Number of qubits: {num_qubits}, Parameter groups: {len(param_structure)}")

        # Step 6: Create parameters matching the exact structure expected by Lambeq
        random.seed(42)
        
        # Build parameters in the exact same structure as _params
        structured_params = []
        for param_group in param_structure:
            if isinstance(param_group, list) and len(param_group) > 0:
                # This parameter group has parameters - create random values for each
                group_values = [random.uniform(0, 2 * np.pi) for _ in param_group]
                structured_params.append(group_values)
            else:
                # Empty parameter group
                structured_params.append([])
            
        if include_debug_prints:
            print("Step 6: Structured parameters generated.")
            print(f"Structured params: {[len(group) for group in structured_params]}")
            
        # Step 7: Define and execute the QNode
        dev = qml.device("default.qubit", wires=num_qubits)

        @qml.qnode(dev)
        def get_vector():
            # Get the circuit function from the PennyLane object
            circuit_func = temp_qml_circuit.make_circuit()
            
            # Pass the structured parameters
            if any(len(group) > 0 for group in structured_params):
                circuit_func(structured_params)
            else:
                circuit_func([])
                
            return qml.state()

        state_vector = get_vector()
        if include_debug_prints:
            print("Step 7: QNode executed successfully.")
            print(f"State vector shape: {state_vector.shape}")
            print(f"State vector (first few elements): {state_vector[:min(8, len(state_vector))]}")
            
        return state_vector, num_qubits

    except Exception as e:
        print(f"CRITICAL ERROR: Failed to process circuit: {e}")
        import traceback
        traceback.print_exc()
        print("Returning a default normalized |0> state to allow program continuation.")
        default_state = np.zeros(2, dtype=np.complex128)
        default_state[0] = 1.0
        return default_state, 1

In [ ]:
if __name__ == "__main__":
    parser = BobcatParser()
    rewriter = Rewriter(['curry', 'prepositional_phrase', 'determiner'])
    # Increase the complexity to get more interesting quantum states
    ansatz = IQPAnsatz({AtomicType.NOUN: 1, AtomicType.SENTENCE: 1}, n_layers=2, n_single_qubit_params=3)

    print("--- Generating states from sentences ---")

    sentence1 = "Alice loves the dog that Bob purchased."
    sentence2 = "Bob loves the dog that Alice sold."
    sentence3 = "The big cat sleeps peacefully."
    sentence4 = "The small bird sings loudly."

    sentences = [sentence1, sentence2, sentence3, sentence4]
    state_data = {}

    for s_idx, sentence in enumerate(sentences):
        try:
            state_vec, num_qubits = lambeq_sentence_to_state_vector(sentence, ansatz, parser, rewriter, include_debug_prints=True)
            # Fix: Store the actual num_qubits, not hardcoded 1
            state_data[sentence] = (state_vec, num_qubits)
            print(f"Sentence {s_idx+1}: '{sentence}'")
            print(f"Generated state with {num_qubits} qubits")
        except Exception as e:
            print(f"Error processing sentence '{sentence}': {e}")
            state_data[sentence] = (None, None)

    print("\n--- Performing Swap Tests ---")
    
    # Filter out sentences that did not produce valid states
    valid_sentences = [s for s in sentences if state_data[s][0] is not None and state_data[s][1] is not None and state_data[s][1] > 0]

    if not valid_sentences:
        print("No valid multi-qubit states generated. Cannot perform Swap Tests meaningfully.")
    else:
        first_num_qubits = state_data[valid_sentences[0]][1]
        all_same_qubits = all(state_data[s][1] == first_num_qubits for s in valid_sentences)

        if not all_same_qubits:
            print("\nWarning: Not all valid sentences resulted in circuits with the same number of qubits.")
            print("Swap Test requires states to have the same number of qubits.")
            print("Pairs with different qubit counts will be skipped.")
            for s in valid_sentences:
                print(f"  '{s}': {state_data[s][1]} qubits")

        for i in range(len(valid_sentences)):
            for j in range(i, len(valid_sentences)):
                s1 = valid_sentences[i]
                s2 = valid_sentences[j]

                vec1, nq1 = state_data[s1]
                vec2, nq2 = state_data[s2]

                if nq1 == nq2:
                    print(f"\nSwap Test between '{s1}' and '{s2}':")
                    # Fix: Use nq1 (or nq2, they're equal) instead of undefined num_qubits
                    overlap = swap_test(vec1, vec2, nq1)
                    print(f"Estimated squared overlap: {overlap:.4f}")
                    if s1 == s2:
                        print(" (Expected to be close to 1.0 for identical states)")
                else:
                    print(f"\nSkipping Swap Test between '{s1}' ({nq1} qubits) and '{s2}' ({nq2} qubits) due to different qubit counts.")

--- Generating states from sentences ---

--- Debugging: Sentence 'Alice loves the dog that Bob purchased.' ---
Step 1: Sentence parsed to diagram.
Step 2: Diagram rewritten.
Step 3: Diagram normalized.
Step 4: Ansatz applied to create DisCoPy circuit.
Step 5: DisCoPy circuit converted to PennyLane object.
Parameter structure: [[Alice__n_0], [Alice__n_1], [Alice__n_2], [dog__n_0], [dog__n_1], [dog__n_2], [the_n_n_0], [the_n_n_1], [the_n_n_2], [Bob__n_0], [Bob__n_1], [Bob__n_2], [], [], [], [], [purchased._n@n_s_0], [], [], [], [], [], [that_n@s@n.l_n_0], [that_n@s@n.l_n_1], [], [], [], [], [], [loves_n@n_s_0], [], []]
Number of qubits: 5, Parameter groups: 32
Step 6: Structured parameters generated.
Structured params: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0]
Step 7: QNode executed successfully.
State vector shape: (32,)
State vector (first few elements): [1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
Sentence 1: 'Alice 